In [19]:
%matplotlib inline

In [20]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import vaex
import pandas as pd
import dask
import dask.dataframe as dd
import numpy as np
import multiprocessing

In [21]:
N = 10**8

In [22]:
s = np.arange(N).astype(str)
vx_df = vaex.from_arrays(s=s)

In [23]:
file = './data/test.hdf5'
vx_df.export(file, progress=True, shuffle=True)

[########################################]:  100.00% elapsed time  :       32s =  0.5m =  0.0h               
  

In [24]:
vx_df = vaex.open(file)
vx_df.executor.buffer_size = len(vx_df) // (multiprocessing.cpu_count() * 2)
pd_df = vx_df.to_pandas_df()
dd_df = dd.from_pandas(pd_df, npartitions=4)

In [25]:
%%time
pd_df.s.str.capitalize().head()

CPU times: user 20.7 s, sys: 2.73 s, total: 23.4 s
Wall time: 23.4 s


0    94680571
1    62404133
2    56924666
3    80522795
4    22217908
Name: s, dtype: object

In [29]:
%%time
vx_df.evaluate(vx_df.s.str.capitalize())

CPU times: user 7.25 s, sys: 3.22 s, total: 10.5 s
Wall time: 10.4 s


array(['94680571', '62404133', '56924666', ..., '17749823', '56091159',
       '88271125'], dtype=object)

In [28]:
%%time
dd_df.s.str.capitalize().compute().head()

CPU times: user 22.1 s, sys: 3.44 s, total: 25.6 s
Wall time: 25.3 s


0    94680571
1    62404133
2    56924666
3    80522795
4    22217908
Name: s, dtype: object

In [35]:
N = 10**6

In [ ]:
vx_df = vaex.open(file)
vx_df.executor.buffer_size = len(vx_df) // (multiprocessing.cpu_count() * 2)
pd_df = vx_df.to_pandas_df()
dd_df = dd.from_pandas(pd_df, npartitions=4)

In [27]:
def timeit():
    times = []
    for i in range(N):
        s = time.time()
        eval(expr, scope)
        e = time.time()
        times.append(e - s)
    return times

In [34]:
methods = {
	'capitalize': 'df.s.str.capitalize()',
	'cat': 'df.s.str.cat(df.s)',
	'contains': 'df.s.str.contains("9", regex=False)',
	'contains(regex)': 'df.s.str.contains("9", regex=True)',
	'count': 'df.s.str.count("9")',
	'endswith': 'df.s.str.endswith("9")',
	'find': 'df.s.str.find("4")',
	'get': 'df.s.str.get(1)',
	'split+:join': 'df.s.str.split(".").str.join("-")',
	'len': 'df.s.str.len()',
	'ljust': 'df.s.str.ljust(10)',
	'lower': 'df.s.str.lower()',
	'lstrip': 'df.s.str.lstrip("9")',
	'match': 'df.s.str.match("1.*")',
	'pad': 'df.s.str.pad(10)',
	'repeat': 'df.s.str.repeat(2)',
	'replace(:default)': 'df.s.str.replace("123", "321")',
	'replace(:no regex)': 'df.s.str.replace("123", "321", regex=False)',
	'replace(:regex)': 'df.s.str.replace("1?[45]4", "1004", regex=True)',
	'rfind': 'df.s.str.rfind("4")',
	'rjust': 'df.s.str.rjust(10)',
	'rstrip': 'df.s.str.rstrip("9")',
	'slice': 'df.s.str.slice(1, 3)',
	'split': 'df.s.str.split(".")',
	'startswith': 'df.s.str.startswith("9")',
	'strip': 'df.s.str.strip("0")',
	'title': 'df.s.str.title()',
	'upper': 'df.s.str.upper()',
	'zfill': 'df.s.str.zfill(10)'
}

In [ ]:
n = 3
for name, method in methods.items():
    t_vaex = (timeit(vaex_expr, n, scope={'df': df_vaex}))/(10**args.n)